In [11]:
%matplotlib notebook

In [ ]:
len(kowiki_edits[kowiki_edits["last_year_edits"] >= 30])

In [ ]:
plt.pyplot.hist(
    np.log10(kowiki_edits["last_year_edits"]),
    bins = (
        0, np.log10(5), np.log10(10), np.log10(25),
        np.log10(100), np.log10(500), np.log10(1000), 
        np.log10(10000), np.log10(70000))
)
plt.pyplot.title("Yearly edit distribution on the Korean Wikipedia")
plt.pyplot.xlabel("edits per year")
plt.pyplot.ylabel("number of users")
plt.pyplot.show()